In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering  
from sklearn.metrics import adjusted_rand_score

In [75]:
df = pd.read_csv("../data/en.openfoodfacts.org.products.tsv", sep = "\t")

# nutr_vars=["carbohydrates_100g", "proteins_100g", "fat_100g", "sodium_100g", "cholesterol_100g"]
nutr_vars=["carbohydrates_100g", "proteins_100g", "sodium_100g", "fat_100g"]
# nutr_vars=["-sucrose_100g","-glucose_100g","-fructose_100g","-lactose_100g","-maltose_100g","-maltodextrins_100g","starch_100g", "polyols_100g"]
df_macro = df[["countries_en"]+nutr_vars].dropna()

countries= ["United States", "France", "Switzerland", "Germany", "Spain", "United Kingdom"]
df_macro = df_macro[df_macro["countries_en"].isin(countries)]
df_macro["region"] = np.where(df_macro["countries_en"]=="United States", "US", "EU")
# df_macro.to_csv("../data/NA-EU_macronutr_unscaled_clean.csv")

df_macro[nutr_vars] = StandardScaler().fit_transform(df_macro[nutr_vars])

print(df_macro.shape)

print(sum(df_macro["region"]=="US"))

# df_macro.to_csv("../data/NA-EU_macronutr_scaled_clean.csv")

c:\users\samre\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(262577, 6)
169617


### Some stuff with beverages, I kinda abandoned this just ignore it for now

In [57]:
df[df["main_category"]=="en:beverages"][["main_category","countries_en"]+bev_nutr_vars].dropna()["countries_en"].value_counts()

France                                   5700
Germany                                   289
United States                             250
Spain                                     192
Switzerland                               185
                                         ... 
France,French Polynesia,New Caledonia       1
Tunisia                                     1
Germany,Hungary                             1
Egypt,United Kingdom,United States          1
Costa Rica                                  1
Name: countries_en, Length: 189, dtype: int64

In [55]:
bev_nutr_vars = ["sodium_100g", "fat_100g","carbohydrates_100g"]

df_bev=df[df["main_category"]=="en:beverages"][["main_category","countries_en"]+bev_nutr_vars].dropna()
df_bev = df_bev[df_bev["countries_en"].isin(countries)]
df_bev["region"] = np.where(df_bev["countries_en"]=="United States", "US", "EU")
df_bev[bev_nutr_vars] = StandardScaler().fit_transform(df_bev[bev_nutr_vars])
print(df_bev.shape)

df_bev.to_csv("../data/NA-EU_bevs_scaled_clean.csv")

display(df_bev)
print(sum(df_bev["region"]=="US"))

(5950, 6)


,main_category,countries_en,sodium_100g,fat_100g,carbohydrates_100g,region
185,en:beverages,France,-0.066776,-0.248861,-0.023854,EU
190,en:beverages,France,-0.094266,-0.219719,-0.652258,EU
231,en:beverages,France,-0.083174,-0.248861,-0.193211,EU
238,en:beverages,France,-0.094721,-0.219719,-0.179840,EU
249,en:beverages,France,-0.049259,-0.248861,-0.193211,EU
...,...,...,...,...,...,...
355876,en:beverages,France,-0.062897,-0.248861,-0.202124,EU
355878,en:beverages,France,2.633017,-0.248861,-0.268976,EU
355966,en:beverages,France,-0.094721,-0.248861,-0.077335,EU
355977,en:beverages,France,0.359902,0.042566,-0.612147,EU


250


# Analysis on Macronutrients by Countries

## KMeans

In [77]:
kmeans = KMeans(2,random_state=69)
kmeans.fit(df_macro[nutr_vars])
print(sum(df_macro["region"]=="US"))

print(np.mean(np.where(df_macro["region"]=="US",0,1)==kmeans.labels_ ))
# print(adjusted_rand_score(kmeans.predict(df_macro[nutr_vars]), np.where(df_macro["region"]=="US",0,1)))

169617
0.6003534201396162


## Hierarchical Agglomerative

(hierarchical agglomerative and spectral clustering both result in memory errors on df_macro)

In [68]:
# del df 
ac = AgglomerativeClustering(n_clusters=2, linkage="ward") 
ac_pred = ac.fit_predict(df_macro[nutr_vars])
print(ac_pred)


[0 1 0 ... 0 0 0]


In [44]:
sc = SpectralClustering(n_clusters=2)
sc_pred = sc.fit_predict(df_macro[nutr_vars])

MemoryError: 